# Shor's algorithm for integer factoring

This example implementation of Shor's algorithm is [adapted from ProjectQ](https://github.com/ProjectQ-Framework/ProjectQ/blob/develop/examples/shor.py).

In [1]:
import math
import random
from fractions import Fraction

try:
    from math import gcd
except ImportError:
    from fractions import gcd

from pyqrack import QrackSimulator, Pauli

import os
os.environ["QRACK_QUNIT_SEPARABILITY_THRESHOLD"] = "0.001"

`toFactor` is the number we are factoring. (You may change it.) 

In [2]:
toFactor=2**60-1

The entire algorithm is implemented below. For the first variant, to factor an integer of `n` qubits or less, we have a footprint of `2n`, but PyQrack's `pown()` is limited in qubit width to single GPU "pages," (usually 2 qubits less than maximum for the GPU,) and the operation is "fully entangling" of the internal representation. 

In [ ]:
%%time
base = int(random.random() * toFactor)
if not gcd(base, toFactor) == 1:
    print("Chose non-relative prime, (without need for quantum computing):")
    print("Factor: {}".format(gcd(base, toFactor)))
else:
    qubitCount = math.ceil(math.log2(toFactor))
    sim = QrackSimulator(2 * qubitCount)

    qi = [i for i in range(qubitCount)]
    qo = [(i + qubitCount) for i in range(qubitCount)]

    # run the quantum subroutine
    for i in qi:
        sim.h(i)
    sim.pown(base, toFactor, qi, qo)
    sim.iqft(qi)
    
    b = [Pauli.PauliZ] * qubitCount
    y = sim.measure_pauli(b, qi)
    r = Fraction(y).limit_denominator(toFactor - 1).denominator

    # try to determine the factors
    if r % 2 != 0:
        r *= 2
    apowrhalf = pow(base, r >> 1, toFactor)
    f1 = gcd(apowrhalf + 1, toFactor)
    f2 = gcd(apowrhalf - 1, toFactor)
    if (not f1 * f2 == toFactor) and f1 * f2 > 1 and int(1.0 * toFactor / (f1 * f2)) * f1 * f2 == toFactor:
        f1, f2 = f1 * f2, int(toFactor / (f1 * f2))
    if f1 * f2 == toFactor and f1 > 1 and f2 > 1:
        print("Factors found : {} * {} = {}".format(f1, f2, toFactor))
    else:
        print("Failed: Found {} and {}".format(f1, f2))

The second implementation requires `2n+2` qubits in footprint to factor a number of `n` qubits or less. However, these methods do not resort to "fully entangled representation," and they are not limited to single GPU "pages."

In [3]:
def MUL(sim, i, a, maxN, qo, qa):
    sim.mcmuln(a, [i], maxN, qo, qa)
    for o in range(len(qa)):
        sim.cswap([i], qa[o], qo[o])
    sim.mcdivn(a, [i], maxN, qo, qa)

def DIV(sim, i, a, maxN, qo, qa):
    sim.mcdivn(a, [i], maxN, qo, qa)
    for o in range(len(qa)):
        sim.cswap([i], qa[o], qo[o])
    sim.mcmuln(a, [i], maxN, qo, qa)

def phase_root_n(sim, n, q):
    sim.mtrx([1, 0, 0, -1**(1.0 / (1<<(n - 1)))], q)

In [7]:
%%time
# Based on https://arxiv.org/abs/quant-ph/0205095
base = int(random.random() * toFactor)
if not gcd(base, toFactor) == 1:
    print("Chose non-relative prime, (without need for quantum computing):")
    print("Factor: {}".format(gcd(base, toFactor)))
else:
    qubitCount = math.ceil(math.log2(toFactor))
    maxN = 1<<qubitCount
    sim = QrackSimulator(3 * qubitCount + 1)

    qo = [i for i in range(qubitCount)]
    qa = [(i + qubitCount) for i in range(qubitCount)]
    qi = 2 * qubitCount
    qm = 2 * qubitCount + 1
    
    m_results = []

    # run the quantum subroutine
    for i in range(qubitCount):
        sim.h(qi)

        # These mul/div operations only work for modulo by powers of 2.
        MUL(sim, qi, i, maxN, qo, qa)
        
        # The above is quasi-successful on its own, even on modulo 2**qubitCount (AKA 1<<qubitCount).
        # Finishing what's below is what's required for strictly literal Shor's factoring algorithm.
        
        # We're implementing the modulo by toFactor, below.
        sim.sub(maxN, qo)
        sim.mcx([qo[-1]], qm)
        sim.mcadd(maxN,[qm], qo)
        DIV(sim, qi, 1<<i, maxN, qo, qa)
        sim.x(qo[-1])
        sim.mcx([qo[-1]], qm)
        sim.x(qo[-1])
        MUL(sim, qi, 1<<i, maxN, qo, qa)
        
        # We use the single control qubit "trick" referenced in Beauregard:
        for j in range(len(m_results)):
            if m_results[j]:
                phase_root_n(sim, j + 2, qi)
        
        m_results.append(sim.m(qi))
        if m_results[-1] == 1:
            sim.x(qi)
        
        sim.h(qi)

    y = 0
    for r in m_results:
        y |= 1<<r
    r = Fraction(y).limit_denominator(toFactor - 1).denominator

    # try to determine the factors
    if r % 2 != 0:
        r *= 2
    apowrhalf = pow(base, r >> 1, toFactor)
    f1 = gcd(apowrhalf + 1, toFactor)
    f2 = gcd(apowrhalf - 1, toFactor)
    if (not f1 * f2 == toFactor) and f1 * f2 > 1 and int(1.0 * toFactor / (f1 * f2)) * f1 * f2 == toFactor:
        f1, f2 = f1 * f2, int(toFactor / (f1 * f2))
    if f1 * f2 == toFactor and f1 > 1 and f2 > 1:
        print("Factors found : {} * {} = {}".format(f1, f2, toFactor))
    else:
        print("Failed: Found {} and {}".format(f1, f2))

Factors found : 5775 * 199640087377809 = 1152921504606846975
CPU times: user 38.8 ms, sys: 0 ns, total: 38.8 ms
Wall time: 38.5 ms
